# Topic Modelling

#### Evan Yathon

This notebook is intended to be run with papermill from the project root.

The purpose of this notebook is to process the review title and content to extract key phrases in each review.  Then key phrases will be used in a regression analysis to find out what is most important for a reviewer in recommending or not recommending the airline.

Topic modelling using LDA will be the tool of choice.

Usage:

`papermill src/ipynbs/topic_modelling.ipynb -p load_path data/cleaned_gw_reviews.csv`

In [1]:
#parameters section for Papermill

load_path = "../../data/cleaned_gw_reviews.csv"

In [10]:
# loading packages

# utils
import pandas as pd
import re

# NLP
import spacy
import gensim 
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim import models
import pyLDAvis.gensim

# display preference for ipynbs
%matplotlib inline

In [3]:
# ensure the dates are parsed correctly with parse_dates argument
reviews = pd.read_csv(load_path, parse_dates = ["date_of_review", "date_flown"])

In [4]:
reviews.head()

,title,review_value,n_user_reviews,reviewer_name,reviewer_country,date_of_review,review_text,aircraft,traveller_type,seat_type,route,date_flown,seat_comfort_rating,cabin_staff_service_rating,food_and_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_for_money_rating,recommendation
0,"""Seat was fine with enough legroom""",7.0,8 reviews,Sander van Kan,Netherlands,2019-07-01,✅ Trip Verified | Dusseldorf to Berlin. Eurowi...,A319,Couple Leisure,Economy Class,Dusseldorf to Berlin,2019-06-01,4.0,3.0,NaN,1.0,3.0,5.0,yes
1,"""crew were smiling and good""",6.0,8 reviews,Sander van Kan,Netherlands,2019-07-01,✅ Trip Verified | Berlin to Dusseldorf. Eurowi...,A319,Couple Leisure,Economy Class,Berlin to Dusseldorf,2019-06-01,3.0,3.0,NaN,1.0,3.0,5.0,yes
2,"""only two agents available""",1.0,6 reviews,Andrew Maynard,United Kingdom,2017-01-04,Check in process at Cologne very poor. Flight ...,NaN,Couple Leisure,Economy Class,CGN to MAN,2017-01-01,2.0,2.0,NaN,NaN,1.0,2.0,no
3,"""good flight and friendly staff""",7.0,1 reviews,T Steen,Netherlands,2016-09-13,✅ Verified Review | Amsterdam to Stuttgart. G...,NaN,Business,Economy Class,AMS to STR,2016-09-01,5.0,5.0,1.0,NaN,5.0,5.0,yes
4,"""never been treated as badly""",1.0,NaN,Karen Kirner,Austria,2016-08-16,✅ Verified Review | I have been a frequent tr...,NaN,Business,Economy Class,DUS to VIE,2016-08-01,1.0,1.0,NaN,NaN,3.0,1.0,no


### Topic Modelling - Preprocessing

Prior to LDA, preprocessing of the text is necessary.  We need to:
- Tokenize each document (review)
- Minimum token length
- Remove stopwords
- Lemmatize
- Consider only specific parts of speech (nouns, verbs etc.)

Use [`spaCy`](https://spacy.io/) for this.

In [5]:
# load the english spacy model
eng_nlp = spacy.load('en')

In [39]:
# create a function to preprocess the reviews and review titles

def preprocess_text(text, min_token_len = 2, relevant_pos = ["NOUN", "VERB", "ADJ"]):
    """
    Given text, min_token_len, and relevant_pos carry out preprocessing of the text 
    and return a preprocessed string. 
    
    Keyword arguments:
    text (str): the text to be preprocessed
    min_token_len (int): min_token_length required
    relevant_pos (list): a list of relevant pos tags
    
    Returns: (str) the preprocessed text
    """
    # use several regex expressions to preprocess strange characters
    # or unwanted text
    
    # remove verified emoji
    text = re.sub(r"✅", "", text)
    
    # remove verified review and trip verified
    text = re.sub(r"Verified Review|Trip Verified", "", text)
    
    # remove anything that is not a word
    text = re.sub(r"[^\w]", " ", text)
    
    # replace multiple spaces with a single space
    text = re.sub(r"\s+", " ", text)
    
    # remove numbers
    text = re.sub(r"[0-9]+", "", text)
    
    # change all text to lowercase
    text = text.lower()
    
    # remove common terms found in topic modelling output
    # eg/ germanwings, flight
    # these words are not useful for this application as they
    # are common terms found in most reviews, and won't be indicative of 
    # things that influenced their recommendation
    terms = ["flight", "germanwing", "fly", "review"]
    text = re.sub("|".join(terms)," ", text)
    
    # utilize spaCy to tokenize and break text into lemmas/POS:
    # - remove stopwords
    # - reduce words to their lemmas
    # - only keep relevant parts of speech
    # - remove short words less than min_token_len
    
    # spacy will utilize
    doc = eng_nlp(text)
    
    processed_text = ""
    
    for token in doc:
        if not token.is_stop and len(token.text) >= min_token_len:# and token.pos_ in relevant_pos:
            processed_text += " " + token.lemma_
    
    return processed_text
    

Use the preprocessing function to process all review texts and review titles.

A good check will be to see if the review topics somewhat make sense with the title topics

In [40]:
reviews["clean_review_text"] = reviews["review_text"].apply(preprocess_text, min_token_len = 3)
reviews["clean_title"] = reviews["title"].apply(preprocess_text, min_token_len = 3)

In [41]:
reviews[["clean_review_text", "review_text", "clean_title", "title"]].head(10)

,clean_review_text,review_text,clean_title,title
0,dusseldorf berlin eurowing operate slight del...,✅ Trip Verified | Dusseldorf to Berlin. Eurowi...,seat fine legroom,"""Seat was fine with enough legroom"""
1,berlin dusseldorf eurowing operate different ...,✅ Trip Verified | Berlin to Dusseldorf. Eurowi...,crew smile good,"""crew were smiling and good"""
2,check process cologne poor display delay minu...,Check in process at Cologne very poor. Flight ...,agent available,"""only two agents available"""
3,amsterdam stuttgart good friendly staff airpl...,✅ Verified Review | Amsterdam to Stuttgart. G...,good friendly staff,"""good flight and friendly staff"""
4,frequent traveler year fly distance cabin cla...,✅ Verified Review | I have been a frequent tr...,treat badly,"""never been treated as badly"""
5,fly london stanste cologne bonn delay hour lo...,✅ Verified Review | Flew Germanwings from Lon...,cramp,"""very cramped"""
6,boyfriend book basic fare hamburg rome fiumic...,My boyfriend and I booked the basic fare from ...,staff friendly,"""staff were friendly"""
7,operation transfer take eurowing read eurowin...,Germanwings operations have now been transferr...,refer eurowing,PLEASE REFER TO EUROWINGS
8,lufthansa operate barcelona berlin tegel figh...,Lufthansa operated by Germanwings from Barcelo...,great value money,"""great value for money"""
9,bangkok london cologne imagine ryanair hour l...,Germanwings from Bangkok to London via Cologne...,pay little money,"""pay very little money"""


It looks mostly good, but I noticed that in the first review it is removing 'enough'.  This could be valuable in context, but I'll keep digging for now.  It's an issue since 'enough' is a stopword and is being removed.

### Topic Modelling: Dictionary and Document-Term Co-occurrence Matrix

In [42]:
# change the cleaned reviews and cleaned titles to 
# an array, required by corpora.Dictionary
review_corpus = [doc.split() for doc in reviews["clean_review_text"].tolist()]
title_corpus = [doc.split() for doc in reviews["clean_title"].tolist()]

In [43]:
# use gensim dictionary to map between words and integer ids
dct_review = corpora.Dictionary(review_corpus)
dct_title = corpora.Dictionary(title_corpus)

In [44]:
# create the document-term co-occurrence matrices
doc_term_mat_review = [dct_review.doc2bow(doc) for doc in review_corpus]
doc_term_mat_title = [dct_title.doc2bow(doc) for doc in title_corpus]

### Topic Modelling:  LDA Topic Model

In [72]:
# create the LDA models
# use a lower alpha, meaning that each document is representative of only a few topics
# do this so that each review doesn't contain similar topics and will have variation
# that we can use to figure out why the recommendation was a yes or no

lda_review = models.LdaModel(corpus = doc_term_mat_review, id2word = dct_review, num_topics = 5, alpha = 0.5)
lda_title = models.LdaModel(corpus = doc_term_mat_title, id2word = dct_title, num_topics = 5, alpha = 0.01)

In [73]:
# view the topics and the words that describe them
lda_review.print_topics()

[(0,
  '0.018*"seat" + 0.014*"good" + 0.012*"time" + 0.010*"crew" + 0.009*"delay" + 0.009*"return" + 0.009*"cologne" + 0.009*"check" + 0.009*"pay" + 0.008*"lufthansa"'),
 (1,
  '0.010*"passenger" + 0.010*"service" + 0.009*"good" + 0.009*"seat" + 0.008*"cgn" + 0.008*"delay" + 0.008*"cologne" + 0.007*"time" + 0.007*"snack" + 0.007*"board"'),
 (2,
  '0.016*"time" + 0.012*"check" + 0.011*"return" + 0.011*"crew" + 0.009*"seat" + 0.009*"cabin" + 0.008*"service" + 0.008*"staff" + 0.008*"luggage" + 0.008*"good"'),
 (3,
  '0.015*"seat" + 0.013*"time" + 0.012*"check" + 0.012*"passenger" + 0.011*"good" + 0.009*"cologne" + 0.009*"delay" + 0.008*"drink" + 0.008*"staff" + 0.008*"pay"'),
 (4,
  '0.010*"service" + 0.009*"board" + 0.008*"late" + 0.008*"hour" + 0.007*"seat" + 0.007*"time" + 0.007*"price" + 0.007*"airline" + 0.007*"delay" + 0.007*"staff"')]

In [74]:
# view the topics and the words that describe them
lda_title.print_topics()

[(0,
  '0.080*"impress" + 0.080*"help" + 0.080*"explanation" + 0.080*"unresponsive" + 0.080*"offer" + 0.080*"customer" + 0.013*"water" + 0.013*"delay" + 0.013*"refuse" + 0.013*"cabin"'),
 (1,
  '0.523*"customer" + 0.056*"friendly" + 0.056*"staff" + 0.029*"crew" + 0.029*"good" + 0.016*"available" + 0.016*"don" + 0.016*"agent" + 0.016*"smile" + 0.016*"satisfied"'),
 (2,
  '0.649*"customer" + 0.027*"eurowing" + 0.027*"cabin" + 0.027*"refer" + 0.027*"book" + 0.027*"vow" + 0.027*"friendly" + 0.027*"crew" + 0.004*"service" + 0.004*"refuse"'),
 (3,
  '0.476*"customer" + 0.035*"choice" + 0.035*"treat" + 0.035*"seat" + 0.035*"legroom" + 0.035*"little" + 0.035*"badly" + 0.035*"airline" + 0.035*"cramp" + 0.035*"fine"'),
 (4,
  '0.561*"customer" + 0.052*"friendly" + 0.036*"professional" + 0.036*"cabin" + 0.036*"delay" + 0.036*"staff" + 0.020*"minute" + 0.020*"money" + 0.020*"problem" + 0.020*"pay"')]

### Topic Modelling: Visualization

In [75]:
# view the reviews 
pyLDAvis.enable_notebook()
vis_reviews = pyLDAvis.gensim.prepare(lda_review, doc_term_mat_review, dct_review, sort_topics=False)
vis_reviews

C:\Users\evany\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.004915 -0.000814       1        1  34.773060
1     -0.014070 -0.007756       2        1  11.532810
2      0.017923  0.016562       3        1  20.788473
3      0.014887 -0.018126       4        1  24.610346
4     -0.013825  0.010134       5        1   8.295313, topic_info=    Category       Freq       Term      Total  loglift  logprob
128  Default  46.000000    service  46.000000  30.0000  30.0000
43   Default  45.000000  passenger  45.000000  29.0000  29.0000
39   Default  31.000000       late  31.000000  28.0000  28.0000
60   Default  39.000000      board  39.000000  27.0000  27.0000
614  Default  28.000000        cgn  28.000000  26.0000  26.0000
61   Default  25.000000       book  25.000000  25.0000  25.0000
74   Default  43.000000      staff  43.000000  24.0000  24.0000
114  Default  18.000000     london  18.000000  23.0000  23.0000
31   Default  31.000000       cost  31.000000  22.0000  22.0000
68   Default  29.000000      offer  29.000000  21.0000  21.0000
41   Default  28.000000    luggage  28.000000  20.0000  20.0000
123  Default  36.000000      price  36.000000  19.0000  19.0000
175  Default  37.000000  lufthansa  37.000000  18.0000  18.0000
38   Default  40.000000       hour  40.000000  17.0000  17.0000
78   Default  28.000000     ticket  28.000000  16.0000  16.0000
22   Default  39.000000    airline  39.000000  15.0000  15.0000
11   Default  40.000000     minute  40.000000  14.0000  14.0000
3    Default  52.000000      delay  52.000000  13.0000  13.0000
117  Default  26.000000        low  26.000000  12.0000  12.0000
73   Default  28.000000      snack  28.000000  11.0000  11.0000
53   Default  78.000000       time  78.000000  10.0000  10.0000
82   Default  32.000000      cabin  32.000000   9.0000   9.0000
97   Default  25.000000    airport  25.000000   8.0000   8.0000
146  Default  13.000000    hamburg  13.000000   7.0000   7.0000
59   Default  11.000000        bad  11.000000   6.0000   6.0000
371  Default   8.000000     second   8.000000   5.0000   5.0000
153  Default  11.000000   customer  11.000000   4.0000   4.0000
29   Default  60.000000      check  60.000000   3.0000   3.0000
34   Default  12.000000       find  12.000000   2.0000   2.0000
79   Default  13.000000      water  13.000000   1.0000   1.0000
..       ...        ...        ...        ...      ...      ...
59    Topic5   1.645284        bad  11.983713   0.5038  -5.7688
22    Topic5   3.818364    airline  39.915550   0.1425  -4.9269
31    Topic5   3.160709       cost  31.135981   0.2019  -5.1159
386   Topic5   1.841575    problem  14.205304   0.4465  -5.6561
117   Topic5   2.772541        low  26.516554   0.2315  -5.2470
114   Topic5   2.129112     london  18.202021   0.3437  -5.5110
11    Topic5   3.507578     minute  40.716454   0.0378  -5.0118
74    Topic5   3.624866      staff  43.156361   0.0125  -4.9789
175   Topic5   3.110018  lufthansa  37.968124  -0.0126  -5.1321
3     Topic5   3.796900      delay  52.761845  -0.1421  -4.9325
68    Topic5   2.637210      offer  29.282642   0.0822  -5.2970
10    Topic5   2.209677    legroom  23.246845   0.1362  -5.4739
402   Topic5   1.206458   possible   8.412624   0.5474  -6.0790
140   Topic5   1.510730     expect  12.895894   0.3452  -5.8541
437   Topic5   1.641453       trip  15.290462   0.2578  -5.7711
53    Topic5   3.876646       time  78.150810  -0.5142  -4.9118
14    Topic5   3.943092       seat  85.356636  -0.5854  -4.8948
146   Topic5   1.528240    hamburg  13.402340   0.3182  -5.8426
199   Topic5   1.661456       euro  16.193199   0.2126  -5.7590
208   Topic5   2.676806        pay  44.030254  -0.3108  -5.2821
127   Topic5   2.817621     return  51.469486  -0.4156  -5.2308
30    Topic5   2.794870    cologne  51.003044  -0.4146  -5.2389
43    Topic5   2.567455  passenger  45.496464  -0.3852  -5.3238
41    Topic5   2.073942    luggage  28.229561  -0.1214  -5.

In [76]:
# view the titles
vis_titles = pyLDAvis.gensim.prepare(lda_title, doc_term_mat_title, dct_title, sort_topics=False)
vis_titles

C:\Users\evany\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.200560  0.012991       1        1   3.379786
1      0.055316  0.054590       2        1  19.692333
2      0.071516 -0.013445       3        1  47.652225
3      0.002608 -0.064185       4        1   5.514703
4      0.071121  0.010049       5        1  23.760952, topic_info=   Category        Freq          Term       Total  loglift  logprob
44  Default  106.000000      customer  106.000000  30.0000  30.0000
40  Default    1.000000          help    1.000000  29.0000  29.0000
27  Default    1.000000       impress    1.000000  28.0000  28.0000
39  Default    1.000000   explanation    1.000000  27.0000  27.0000
26  Default    1.000000  unresponsive    1.000000  26.0000  26.0000
41  Default    1.000000         offer    1.000000  25.0000  25.0000
9   Default    4.000000         staff    4.000000  24.0000  24.0000
18  Default    1.000000        little    1.000000  23.0000  23.0000
22  Default    1.000000        choice    1.000000  22.0000  22.0000
11  Default    1.000000         treat    1.000000  21.0000  21.0000
2   Default    1.000000          seat    1.000000  20.0000  20.0000
1   Default    1.000000       legroom    1.000000  19.0000  19.0000
10  Default    1.000000         badly    1.000000  18.0000  18.0000
21  Default    1.000000       airline    1.000000  17.0000  17.0000
0   Default    1.000000          fine    1.000000  16.0000  16.0000
12  Default    1.000000         cramp    1.000000  15.0000  15.0000
20  Default    2.000000  professional    2.000000  14.0000  14.0000
31  Default    2.000000         delay    2.000000  13.0000  13.0000
8   Default    6.000000      friendly    6.000000  12.0000  12.0000
4   Default    1.000000          good    1.000000  11.0000  11.0000
13  Default    2.000000      eurowing    2.000000  10.0000  10.0000
14  Default    2.000000         refer    2.000000   9.0000   9.0000
36  Default    2.000000           vow    2.000000   8.0000   8.0000
35  Default    2.000000          book    2.000000   7.0000   7.0000
25  Default    4.000000         cabin    4.000000   6.0000   6.0000
43  Default    1.000000        minute    1.000000   5.0000   5.0000
32  Default    1.000000       problem    1.000000   4.0000   4.0000
19  Default    1.000000           pay    1.000000   3.0000   3.0000
42  Default    1.000000      heathrow    1.000000   2.0000   2.0000
16  Default    1.000000         money    1.000000   1.0000   1.0000
..      ...         ...           ...         ...      ...      ...
12   Topic5    0.144125         cramp    1.076531  -0.5737  -5.7203
0    Topic5    0.144125          fine    1.076531  -0.5737  -5.7203
21   Topic5    0.144125       airline    1.076531  -0.5737  -5.7203
10   Topic5    0.144125         badly    1.076532  -0.5737  -5.7203
1    Topic5    0.144125       legroom    1.076532  -0.5737  -5.7203
2    Topic5    0.144125          seat    1.076532  -0.5737  -5.7203
11   Topic5    0.144125         treat    1.076532  -0.5737  -5.7203
22   Topic5    0.144125        choice    1.076532  -0.5737  -5.7203
41   Topic5    0.144125         offer    1.193305  -0.6767  -5.7203
26   Topic5    0.144125  unresponsive    1.193306  -0.6767  -5.7203
39   Topic5    0.144125   explanation    1.193306  -0.6767  -5.7203
27   Topic5    0.144125       impress    1.193306  -0.6767  -5.7203
40   Topic5    0.144125          help    1.193306  -0.6767  -5.7203
15   Topic5    0.144125         great    1.262210  -0.7328  -5.7203
34   Topic5    0.144125          late    1.262211  -0.7328  -5.7203
17   Topic5    0.144125         value    1.262211  -0.7328  -5.7203
30   Topic5    0.144125       service    1.262211  -0.7328  -5.7203
28   Topic5    0.144125        refuse    1.262211  -0.7328  -5.7203
37   Topic5    0.144125          food    1.262211  -0.7328  -5.7203
38   Topic5    0.144125      terrible    1.262211  -0.7328  -5.7203
24   Topic5    0.144125          info    1.